In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 'TC_AUC',0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3,True

In [ ]:
import os
data_path = '../data/'
origin_path = data_path + 'origin_data/'

op_train_new_fn = 'operation_train_new.csv'
tran_train_new_fn = 'transaction_train_new.csv'
tag_train_new_fn = 'tag_train_new.csv'
op_train_sorted_fn = 'op_train_sorted.csv'
tran_train_sorted_fn = 'tran_train_sorted.csv'
tag_train_sorted_fn = 'tag_train_sorted.csv'

op_origin_fn = 'op_origin.csv'
tran_origin_fn = 'tran_origin.csv'


op_train_new_file = data_path + op_train_new_fn
tran_train_new_file = data_path + tran_train_new_fn
tag_train_new_file = data_path + tag_train_new_fn
op_train_sorted_file = data_path + op_train_sorted_fn
tran_train_sorted_file = data_path + tran_train_sorted_fn
tag_train_sorted_file = data_path + tag_train_sorted_fn

op_origin_file = origin_path + op_origin_fn
tran_origin_file = origin_path + tran_origin_fn

In [ ]:
def load_data():
    print('[info]:start read from op_train...')
    op_train = pd.read_csv(op_origin_file).drop('Unnamed: 0', axis=1)
    print('[info]:start read from tran_train...')
    tran_train = pd.read_csv(tran_origin_file).drop('Unnamed: 0', axis=1)
    print('[info]:start read from tag_train...')
    tag_train = pd.read_csv(tag_train_sorted_file).drop('Unnamed: 0', axis=1)
    return op_train, tran_train, tag_train

In [ ]:
op_train, tran_train, tag_train = load_data()

In [ ]:
op_gb = op_train.groupby('UID')
drop_fts = ['mode','success','time','device1','device2','device_code1','device_code2','device_code3','mac1','ip1','ip2','ip1_sub','ip2_sub','timestamp']
op_train_nf = op_gb.count().drop(drop_fts, axis='columns')
op_train_nf.rename(columns={'day':'day_cnts'}, inplace=True)

In [ ]:
op_train_nf['top_appear_day']

In [ ]:
for index, row in data.iterrows():
op_gb['day'].value_counts()

In [ ]:
op_train_nf = op_train_nf.join(day_cnt)
op_train_nf

In [ ]:
def get_feature(op,trans,label):
    for feature in op.columns[2:]:
        label = label.merge(op.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
        label =label.merge(op.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
    
    for feature in trans.columns[2:]:
        if trans_train[feature].dtype == 'object':
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
        else:
            print(feature)
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].max().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].min().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].sum().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].mean().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].std().reset_index(),on='UID',how='left')
    return label

In [ ]:
train = get_feature(op_train,trans_train,y)
test = get_feature(op_test,trans_test,sub)

In [ ]:
list(train.columns)

In [ ]:
train.head()

In [ ]:
train = train.fillna(-1)
test = test.fillna(-1)

In [ ]:
train = train.drop(['UID','Tag'],axis = 1).fillna(-1)
label = y['Tag']

In [ ]:
test_id = test['UID']
test = test.drop(['UID','Tag'],axis = 1).fillna(-1)